In [1]:
sc

In [4]:
import csv

nypd_collisions = sc.textFile('nypd_carl2_part0.csv')
weather_data = sc.textFile('processed_2011-2017_fixed.csv')

In [5]:
# list(enumerate(nypd_collisions.first().split(',')))

In [6]:
list(enumerate(weather_data.first().split(',')))

[(0, u''),
 (1, u'Conditions'),
 (2, u'DateUTC'),
 (3, u'DatetimeEDT'),
 (4, u'Dew_PointF'),
 (5, u'Events'),
 (6, u'Freezing'),
 (7, u'Gust_SpeedMPH'),
 (8, u'Humidity'),
 (9, u'LowVisibility'),
 (10, u'PrecipitationIn'),
 (11, u'Rain'),
 (12, u'Sea_Level_PressureIn'),
 (13, u'Snow'),
 (14, u'TemperatureF'),
 (15, u'TimeEDT'),
 (16, u'VisibilityMPH'),
 (17, u'WindDirDegrees'),
 (18, u'Wind_Direction'),
 (19, u'Wind_SpeedMPH'),
 (20, u'Windy')]

In [9]:
import csv
import datetime
def map_weather_hours(idx, part):
    if idx == 0:
        part.next()
    for p in csv.reader(part):
        datehour= p[3].split(':')[0]
        yield (datehour, p)

weather_data_rdd = weather_data.mapPartitionsWithIndex(map_weather_hours)
# sorted(weather_data_rdd.groupByKey().mapValues(list).collect())
weather_data_rdd = weather_data_rdd.groupByKey().mapValues(list)

In [10]:
import csv
from dateutil.parser import parse

def map_collision_hours(idx, part):
    if idx == 0:
        part.next()
    for p in csv.reader(part):
        datehour = str(parse(p[7])).split(':')[0]
        p[7] = str(parse(p[7]))
        yield (datehour, p)

nypd_collisions_rdd = nypd_collisions.mapPartitionsWithIndex(map_collision_hours)
# nypd_collisions_rdd.take(5)

In [11]:
joined_data = nypd_collisions_rdd.join(weather_data_rdd)

In [42]:
# joined_data_collected = joined_data.collect()

In [8]:
# joined_data_collected[0]

In [9]:
# x = joined_data.take(1)[0]
# datehour, tupled_lists = x[0], x[1]
# tuple_weather = tupled_lists[1]
# tuple_weather[0]
# x[1][1][0]

In [12]:
from pyspark.sql import Row
final_join=joined_data.map(lambda x: x[1][0] + x[1][1][0])

In [13]:
from pyspark.sql.types import StructType, StructField, StringType
schema = StructType([StructField('BOROUGH', StringType()),
                    StructField('CONTRIBUTING_FACTOR_VEHICLE_1', StringType()),
                    StructField('CONTRIBUTING_FACTOR_VEHICLE_2', StringType()),
                    StructField('CONTRIBUTING_FACTOR_VEHICLE_3', StringType()),
                    StructField('CONTRIBUTING_FACTOR_VEHICLE_4', StringType()),
                    StructField('CONTRIBUTING_FACTOR_VEHICLE_5', StringType()),
                    StructField('CROSS_STREET_NAME', StringType()),
                    StructField('DATETIME', StringType()),
                    StructField('LATITUDE', StringType()),
                    StructField('LOCATION', StringType()),
                    StructField('LONGITUDE', StringType()),
                    StructField('NUMBER_OF_CYCLIST_INJURED', StringType()),
                    StructField('NUMBER_OF_CYCLIST_KILLED', StringType()),
                    StructField('NUMBER_OF_MOTORIST_INJURED', StringType()),
                    StructField('NUMBER_OF_MOTORIST_KILLED', StringType()),
                    StructField('NUMBER_OF_PEDESTRIANS_INJURED', StringType()),
                    StructField('NUMBER_OF_PEDESTRIANS_KILLED', StringType()),
                    StructField('NUMBER_OF_PERSONS_INJURED', StringType()),
                    StructField('NUMBER_OF_PERSONS_KILLED', StringType()),
                    StructField('OFF_STREET_NAME', StringType()),
                    StructField('ON_STREET_NAME', StringType()),
                    StructField('UNIQUE_KEY', StringType()),
                    StructField('VEHICLE_TYPE_CODE_1', StringType()),
                    StructField('VEHICLE_TYPE_CODE_2', StringType()),
                    StructField('VEHICLE_TYPE_CODE_3', StringType()),
                    StructField('VEHICLE_TYPE_CODE_4', StringType()),
                    StructField('VEHICLE_TYPE_CODE_5', StringType()),
                    StructField('ZIP_CODE', StringType()),
                    StructField('', StringType()),
                    StructField('Conditions', StringType()),
                    StructField('DateUTC', StringType()),
                    StructField('DatetimeEDT', StringType()),
                    StructField('Dew_PointF', StringType()),
                    StructField('Events', StringType()),
                    StructField('Freezing', StringType()),
                    StructField('Gust_SpeedMPH', StringType()),
                    StructField('Humidity', StringType()),
                    StructField('LowVisibility', StringType()),                     
                    StructField('PrecipitationIn', StringType()),
                    StructField('Rain', StringType()),
                    StructField('Sea_Level_PressureIn', StringType()),
                    StructField('Snow', StringType()),
                    StructField('TemperatureF', StringType()),
                    StructField('TimeEDT', StringType()),
                    StructField('VisibilityMPH', StringType()),
                    StructField('WindDirDegrees', StringType()),
                    StructField('Wind_Direction', StringType()),
                    StructField('Wind_SpeedMPH', StringType()),
                    StructField('Windy', StringType())])

In [14]:
final_join_df = sqlContext.createDataFrame(final_join, schema)

In [15]:
# final_join_df.write.csv('joined_weather_collisions')
# final_join_df.coalesce(1).write.csv('coalesce_joined_weather_collisions2', header=True)
sorted_final_join = final_join_df.sort('DATETIME')
sorted_final_join.take(2)

[Row(BOROUGH=u'QUEENS', CONTRIBUTING_FACTOR_VEHICLE_1=u'Unspecified', CONTRIBUTING_FACTOR_VEHICLE_2=u'Unspecified', CONTRIBUTING_FACTOR_VEHICLE_3=u'', CONTRIBUTING_FACTOR_VEHICLE_4=u'', CONTRIBUTING_FACTOR_VEHICLE_5=u'', CROSS_STREET_NAME=u'70 DRIVE', DATETIME=u'2016-06-20 21:06:00', LATITUDE=u'', LOCATION=u'', LONGITUDE=u'', NUMBER_OF_CYCLIST_INJURED=u'0', NUMBER_OF_CYCLIST_KILLED=u'0', NUMBER_OF_MOTORIST_INJURED=u'0', NUMBER_OF_MOTORIST_KILLED=u'0', NUMBER_OF_PEDESTRIANS_INJURED=u'0', NUMBER_OF_PEDESTRIANS_KILLED=u'0', NUMBER_OF_PERSONS_INJURED=u'0', NUMBER_OF_PERSONS_KILLED=u'0', OFF_STREET_NAME=u'', ON_STREET_NAME=u'METROPOLITAN AVENUE', UNIQUE_KEY=u'3465402', VEHICLE_TYPE_CODE_1=u'PASSENGER VEHICLE', VEHICLE_TYPE_CODE_2=u'PASSENGER VEHICLE', VEHICLE_TYPE_CODE_3=u'', VEHICLE_TYPE_CODE_4=u'', VEHICLE_TYPE_CODE_5=u'', ZIP_CODE=u'11375', =u'58671', Conditions=u'Clear', DateUTC=u'2016-06-21 01:51:00', DatetimeEDT=u'2016-06-20 21:51:00', Dew_PointF=u'63.0', Events=u'', Freezing=u'0', Gu

In [16]:
sorted_final_join.coalesce(1).write.csv('coalesce_joined_weather_collisions_sorted_fixed', header=True)

In [18]:
# len(final_join_df.columns)

In [19]:
# list((nypd_collisions.first().split(',')))

In [20]:
# list((weather_data.first().split(',')))

In [108]:
# final_join.coalesce(1,True).saveAsTextFile('coalesce_join_collisions_weather')

In [17]:
# final_join_df.coalesce(1).write.csv('coalesce_joined_weather_collisions2', header=True)